In [1]:
class Vendas():
    def __init__(self, regiao, estado, municipio, revenda, 
                 produto, dt_coleta, valor_compra, 
                 valor_venda, unidade_medida, bandeira):
        self.regiao = regiao
        self.estado = estado
        self.municipio = municipio
        self.revenda = revenda
        self.produto = produto
        self.dt_coleta = dt_coleta
        #self.mes = mes
        #self.ano = ano
        self.valor_compra = valor_compra
        self.valor_venda = valor_venda
        self.unidade_medida = unidade_medida
        self.bandeira = bandeira

    def get_regiao(self):
        return self.regiao

    def get_estado(self):
        return self.estado

    def get_municipio(self):
        return self.municipio

    def get_revenda(self):
        return self.revenda

    def get_produto(self):
        return self.produto    
        
    def get_dt_coleta(self):
        return self.dt_coleta
    
    def get_mes(self):
        return self.mes
    
    def get_ano(self):
        return self.ano
    
    def get_valor_compra(self):
        return self.valor_compra    
    
    def get_unidade_medida(self):
        return self.unidade_medida        
    
    def get_bandeira(self):
        return self.bandeira     

In [2]:
def load_vendas_info_file(fname):
    vendas = []
    with open(fname, encoding="latin-1") as f:
        next(f)
        for line in f: 
            vendas.append(Vendas(*str(line).strip().split('\t')))
    f.close()
    return vendas

In [3]:
vendas_2016_2 = load_vendas_info_file('2016-2_CA.csv')
#vendas_2017_1 = load_vendas_info_file('2017-1_CA.csv')
#vendas_2017_2 = load_vendas_info_file('2017-2_CA.csv')
#vendas_2018_1 = load_vendas_info_file('2018-1_CA.csv')

In [4]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

CASSANDRA_USER = 'cassandra'
CASSANDRA_PASSWD = 'password'

def create_connection(nodeIP, keyspaceName):
    """
    Function to create a connection object
    """

    client = Connection()
    client.connect([nodeIP]) #Observar que o nodeIP é uma lista e não somente um único valor
    client.set_keyspace(keyspaceName)
    return client


class Connection:
    session = None
    keyspace = ""

    def connect(self, nodes):
        auth_provider = PlainTextAuthProvider(username=str(CASSANDRA_USER), password=str(CASSANDRA_PASSWD))
        cluster = Cluster(nodes, auth_provider=auth_provider)
        metadata = cluster.metadata
        self.session = cluster.connect()

    def get_session():
        return self.session

    def set_keyspace(self, keyspace):
        self.keyspace = keyspace

    def get_keyspace(self):
        return self.keyspace

    def close(self):
        self.session.cluster.shutdown()
        self.session.shutdown()

    def query_schema(self, table):
        table_keyspace = self.keyspace+"."+table
        query = " SELECT * FROM "+ table_keyspace +";"
        result_set = self.session.execute(query)
        return result_set

In [5]:
#Classe pai das conexões
class CommonCassandraDB():

    def __init__(self):
        self.client = None

    def set_cassandra_client(self, nodeIP, keyspaceName):
        self.client = create_connection(nodeIP, keyspaceName)

    def close_connection(self):
        self.client.close()

In [6]:
class Vendas_cassandra(CommonCassandraDB):
    
    def __init__(self, nodeIP, keyspace):
        self.cassandra_table_reference = 'vendas' #nome da Tabela do Cassandra

        self.set_cassandra_client(nodeIP, keyspace)
    
    def get_full_table(self):
        table = self.client.get_keyspace() + "." + self.cassandra_table_reference
        sql_query = """
                    SELECT *
                    FROM @$$$$$$$$$$$$@
                """
        sql_query = sql_query.replace("@$$$$$$$$$$$$@", table)
        result_set = self.client.session.execute(sql_query)
        return result_set 

    def show_row(self, row):
        print(row.get_regiao()) 
        print(row.get_estado())
        print(row.get_municipio()) 
        print(row.get_revenda())
        print(row.get_produto())
        print(row.get_dt_coleta())
        print(row.get_mes())
        print(row.get_ano())
        print(row.get_valor_compra())
        print(row.get_valor_venda())
        print(row.get_unidade_medida())
        print(row.get_bandeira())

    def insert(self, venda):
        try:
            table = self.client.get_keyspace() + "." + 'vendas'
            sql_query = """
            INSERT INTO @$$$$$$$$$$$$@
            (
                regiao, 
                estado, 
                municipio, 
                revenda, 
                produto, 
                dt_coleta, 
                mes,
                ano,
                valor_compra, 
                valor_venda, 
                unidade_medida, 
                bandeira
            )
            VALUES (?,?,?,?,?,?,?,?,?,?,?,?);
            """
            sql_query = sql_query.replace("@$$$$$$$$$$$$@", table)
            bound_statement = self.client.session.prepare(sql_query)
            self.client.session.execute( bound_statement.bind((
                    venda.get_regiao(),
                    venda.get_estado(),
                    venda.get_municipio(),
                    venda.get_revenda(),
                    venda.get_produto(),
                    venda.get_dt_coleta(),
                    venda.get_mes(),
                    venda.get_ano(),
                    venda.get_valor_compra(), 
                    venda.get_valor_venda(), 
                    venda.get_unidade_medida(), 
                    venda.get_bandeira()                
                    ))
                )
            return True
        except:
            raise
        return False


In [ ]:
connect = CommonCassandraDB()

set_cassandra_client(self, nodeIP, keyspaceName) 
    
    
Vendas_cassandra()